### Purpose
A simple script that takes a model checkpoint (if saved in said format), builds model, and saves. Can allow for less processing in downstream scripts.

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import tensorflow as tf

import IPython.display as display
from PIL import Image
import numpy as np
import os
import pathlib

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.__version__

'2.14.0'

In [12]:
# trained model chkpt
checkpoint_path = "../results/ABGQI-CNN/cp.ckpt"

# output model
out_file = '../results/ABGQI-CNN'

# target classes ABGIQ
n_classes = 5

# Prepare orginal model to load weights

In [13]:
# image parameters
batch_size = 32

# dimensions of our images
img_width, img_height, img_depth = 224, 224, 3 # 224 pixels x 224 pixels x 3 bands (RGB)
IMG_SHAPE = (img_width, img_height, img_depth)

# Create the base model from the pre-trained model
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet',
                                               alpha = 1.4)

In [14]:
image_batch = tf.constant(np.zeros((batch_size, img_width, img_height, img_depth))) # instantiate array of n images by image size
feature_batch = base_model(image_batch) # specify model batch
base_model.trainable = False # don't train
print(feature_batch.shape)

(32, 7, 7, 1792)


In [15]:
# build model architecture, 2D GAP layer accounts for the 2D spectrogram shape
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 1792)


In [16]:
# specify model layers, classes, and prediction
prediction_layer = tf.keras.layers.Dense(n_classes)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 5)


In [17]:
# build model
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [18]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),         #Whether to interpret y_pred as a tensor of logit values. By default, we assume that y_pred contains probabilities (i.e., values in [0, 1]). **Note - Using from_logits=True may be more numerically stable.
              metrics=['accuracy'])

In [19]:
model.load_weights(checkpoint_path)

NotFoundError: ignored

In [ ]:
# Orginal model
model.summary()

# Model output

In [ ]:
model.save(out_file)